In [1]:
import pandas as pd
import regex as re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from Cleansing import clean
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score


In [2]:
train = pd.read_csv('dataset/train_preprocess.tsv.txt', names=['Kalimat','Sentiment'], sep='\t')
train.head()

,Kalimat,Sentiment
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive
1,mohon ulama lurus dan k212 mmbri hujjah partai...,neutral
2,lokasi strategis di jalan sumatera bandung . t...,positive
3,betapa bahagia nya diri ini saat unboxing pake...,positive
4,duh . jadi mahasiswa jangan sombong dong . kas...,negative


In [3]:
train['new_clean'] = train.Kalimat.apply(clean)

**EDA**

In [5]:
train['total_word'] = train['new_clean'].apply(lambda x: len(x.split()))
train['total_char'] = train['new_clean'].apply(len)
train.head()

,Kalimat,Sentiment,new_clean,total_word,total_char
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive,warung dimiliki pengusaha pabrik puluhan terke...,31,242
1,mohon ulama lurus dan k212 mmbri hujjah partai...,neutral,mohon ulama lurus k212 mmbri hujjah partai diw...,12,72
2,lokasi strategis di jalan sumatera bandung . t...,positive,lokasi strategis jalan sumatra bandung nyaman ...,19,120
3,betapa bahagia nya diri ini saat unboxing pake...,positive,betapa bahagia unboxing paket barang bagus men...,8,58
4,duh . jadi mahasiswa jangan sombong dong . kas...,negative,aduh mahasiswa sombong kasih kartu kuning bela...,15,110


In [6]:
train.dtypes

Kalimat       object
Sentiment     object
new_clean     object
total_word     int64
total_char     int64
dtype: object

In [7]:
train.isna().sum()

Kalimat       0
Sentiment     0
new_clean     0
total_word    0
total_char    0
dtype: int64

In [8]:
train.describe()

,total_word,total_char
count,11000.000000,11000.000000
mean,15.484000,106.707545
std,9.609236,66.374186
min,0.000000,0.000000
25%,8.000000,56.000000
50%,13.000000,93.000000
75%,22.000000,151.000000
max,66.000000,422.000000


**Feature Extraction & Build Modelling**

In [9]:
# Separating X (Predictor) and Y (target) variabel
y = train["Sentiment"]
xraw = train["new_clean"]

In [10]:
#Converting text to vector using Sklearn TfidfVectorizer
vec = TfidfVectorizer()
x = vec.fit_transform(xraw)

In [11]:
#covert X to regular array
x = x.toarray()

In [12]:
#Train test split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [13]:
#Build & training model
model = LogisticRegression(max_iter=11000)
model.fit(x_train, y_train)

LogisticRegression(max_iter=11000)

In [18]:
#Evaluating our model
score = model.score(x_test, y_test)
print('Akurasi dari prediksi sentimen teks ini mencapai',score*100)

Akurasi dari prediksi sentimen teks ini mencapai 83.04545454545455


In [19]:
test = pd.read_csv("dataset/sample.csv", encoding='latin-1')
test.head()

,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong
0,- disaat semua cowok berusaha melacak perhatia...,1,1,1,0,0,0,0,0,1,1,0,0
1,RT USER: USER siapa yang telat ngasih tau elu?...,0,1,0,0,0,0,0,0,0,0,0,0
2,"41. Kadang aku berfikir, kenapa aku tetap perc...",0,0,0,0,0,0,0,0,0,0,0,0
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,0,0,0,0,0,0,0,0,0,0,0,0
4,USER USER Kaum cebong kapir udah keliatan dong...,1,1,0,1,1,0,0,0,0,0,1,0


In [20]:
x_test = vec.transform(test['Tweet'])
prediction = model.predict(x_test.toarray())
prediction

array(['positive', 'negative', 'positive', ..., 'negative', 'negative',
       'negative'], dtype=object)